# Verification de l'implementation de la correlation de Nusselt pour le flux en paroi

In [ ]:
from trustutils import run

run.TRUST_parameters()
run.reset()

lx, ly, lz = 1e-1, 5e-3, 1
perimeter = 2 * (lx + ly)
sec = lx * ly
dh = 4 * sec / perimeter
nx, ny, nz = 2, 2, 100
rho, mu, k, cp = 2500, 1e-2, 1, 500
flow_rate = 0.1
velocity = flow_rate / sec / rho
Tin = 0
Tw = 100
Re = rho * dh * velocity / mu
Pr = cp * mu / k

parameters = {
    "lx": lx,
    "ly": ly,
    "lz": lz,
    "xmid": 0.5 * lx,
    "ymid": 0.5 * ly,
    "nx": nx,
    "ny": ny,
    "nz": nz,
    "rho": rho,
    "mu": mu,
    "lambda": k,
    "cp": cp,
    "dh": dh,
    "m": flow_rate,
    "v": velocity,
    "Tin" : Tin,
    "Tw" : Tw,
    "P" : perimeter,
    "dh" : dh,
    "tmax" : lz / velocity * 2
}

def Nu_cst(Re, Pr):
    return 4

def Nu_DB(Re, Pr):
    return 0.023 * Re**0.8 * Pr**0.4

nusselts = [("constant", "4", Nu_cst), ("Dittus-Boetler", "0.023*Re^0.8*Pr^0.4", Nu_DB)]
dis = ["PolyMAC_P0"]
pbs = ["std", "multi"]

for pb in pbs:
    for d in dis:
        for Nu_name, Nu_str, Nu_f in nusselts:
            h = k * Nu_f(Re, Pr) / dh
            poly = "" if d == "VDF" else "Polyedriser dom"
            run.addCaseFromTemplate(f"jdd_{pb}.data", f"{d}/{Nu_name}", {**parameters, **{"dis": d, "poly": poly, "nusselt": Nu_str, "h" : h}})
run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a = plot.Graph()
for pb in pbs:
    for d in dis:
        for Nu_name, Nu_str, Nu_f in nusselts:
            a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/{Nu_name}/jdd_{pb}.dt_ev", label=f"{d} {pb} {Nu_name}")

a.scale(yscale="log")

In [ ]:
for pb in pbs:
    a = plot.Graph(pb)
    for d in dis:
        for Nu_name, Nu_str, Nu_f in nusselts:
            a.addSegment(
                f"{run.BUILD_DIRECTORY}/{d}/{Nu_name}/jdd_{pb}_T.son",
                compo=0,
                marker="--",
                markersize=12,
                label=f"{Nu_name} - {d}",
            )
            if (d == "PolyMAC_P0"):
                a.addSegment(
                f"{run.BUILD_DIRECTORY}/{d}/{Nu_name}/jdd_{pb}_T_EXACT.son",
                compo=0,
                marker="-",
                markersize=12,
                label=f"{Nu_name} (exact)",
            )
a.legend()
a.label("z [m]", "Temperature")